In [1]:
import pandas as pd
import pepfeature
from string import ascii_lowercase as alc
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('./peptides-complete.csv')

/tmp/ipykernel_161349/1422333858.py:1: DtypeWarning: Columns (16,17,23,24,31,32,38,53,54,57,63,64,65,66,67,68,71,76,79) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./peptides-complete.csv')


In [3]:
drop_list = ['NAME', 'SOURCE GENE - DB LINK', 'SOURCE GENE - NOTE', 'PDB NAME', 'PDB LINK', 'PDB FILE LINK', 'TARGET ACTIVITY - TARGET SPECIES',
       'TARGET ACTIVITY - ACTIVITY MEASURE GROUP',
       'TARGET ACTIVITY - ACTIVITY MEASURE VALUE',
       'TARGET ACTIVITY - CONCENTRATION', 'TARGET ACTIVITY - UNIT',
       'TARGET ACTIVITY - PH', 'TARGET ACTIVITY - IONIC STRENGTH',
       'TARGET ACTIVITY - SALT TYPE', 'TARGET ACTIVITY - MEDIUM',
       'TARGET ACTIVITY - CFU', 'TARGET ACTIVITY - CFU GROUP',
       'TARGET ACTIVITY - NOTE',
       'TARGET ACTIVITY - ACTIVITY (μg/ml) (Calculated By DBAASP)',
       'TARGET ACTIVITY - REFERENCE','INTERCHAIN BOND - NOTE','SYNERGY - ACTIVITY MEASURE GROUP', 'SYNERGY - UNIT',
       'SYNERGY - PEPTIDE ID', 'SYNERGY - ANTIBIOTIC ID',
       'SYNERGY - ANTIBIOTIC NAME', 'SYNERGY - PEPTIDE ACTIVITY',
       'SYNERGY - PEPTIDE ANTIBIOTIC ACTIVITY',
       'SYNERGY - ANTIBIOTIC ACTIVITY',
       'SYNERGY - ANTIBIOTIC PEPTIDE ACTIVITY', 'SYNERGY - FICI',
       'SYNERGY - REFERENCE', 'UNIRPROT - ID', 'UNIRPROT - DESCRIPTION',
       'UNIRPROT - PRO PEPTIDE', 'UNIRPROT - URL', 'ARTICLES - JOURNAL',
       'ARTICLES - YEAR', 'ARTICLES - VOLUME', 'ARTICLES - PAGES',
       'ARTICLES - TITLE', 'ARTICLES - ADDITIONAL', 'ARTICLES - PUBMED',
       'ARTICLES - AUTHORS', 'TARGET GROUP', 'TARGET OBJECT',
       'SOURCE GENE - KINGDOM', 'SOURCE GENE - SOURCE',
       'SOURCE GENE - SUBKINGDOM', 'SOURCE GENE - GENE',
       'SOURCE GENE - GENE IN SEQUENCE', 'INTERCHAIN BOND - CHAIN 1',
       'INTERCHAIN BOND - CHAIN 2', 'INTERCHAIN BOND - CHAIN 3',
       'INTERCHAIN BOND - CHAIN 4', 'INTERCHAIN BOND - BOND',
       'INTRACHAIN BOND - POSITION 1', 'INTRACHAIN BOND - POSITION 2',
       'INTRACHAIN BOND - BOND', 'INTRACHAIN BOND - NOTE',
       'UNUSUAL OR MODIFIED AMINO ACID - POSITION',
       'UNUSUAL OR MODIFIED AMINO ACID - MODIFICATION TYPE',
       'UNUSUAL OR MODIFIED AMINO ACID - BEFORE MODIFICATION',
       'UNUSUAL OR MODIFIED AMINO ACID - NOTE','HEMOLITIC CYTOTOXIC ACTIVITY - NOTE',
       'HEMOLITIC CYTOTOXIC ACTIVITY - REFERENCE', 'SYNERGY - TARGET SPECIE','HEMOLITIC CYTOTOXIC ACTIVITY - PH',
       'HEMOLITIC CYTOTOXIC ACTIVITY - IONIC STRENGTH',
       'HEMOLITIC CYTOTOXIC ACTIVITY - SALT TYPE', 'HEMOLITIC CYTOTOXIC ACTIVITY - ACTIVITY (μg/ml) (Calculated By DBAASP)',
       'SYNTHESIS TYPE',]

df = pd.read_csv('./peptides-complete.csv', index_col=False)

df = df.drop(drop_list, axis=1)

/tmp/ipykernel_161349/1613924343.py:37: DtypeWarning: Columns (16,17,23,24,31,32,38,53,54,57,63,64,65,66,67,68,71,76,79) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./peptides-complete.csv', index_col=False)


In [4]:
# display(df)

In [5]:
df = df[df['HEMOLITIC CYTOTOXIC ACTIVITY - TARGET CELL'] == 'Human erythrocytes']
df = df[df['COMPLEXITY'] == 'Monomer']
df = df[df['HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS GROUP'] != 'MHC']
df = df.dropna(subset=['HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS VALUE', 'HEMOLITIC CYTOTOXIC ACTIVITY - CONCENTRATION'])
# df['SYNTHESIS TYPE'] = df['SYNTHESIS TYPE'].astype('category')
for lower_case in alc:
    df = df[df['SEQUENCE'].str.contains(lower_case) == False]

df['HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS VALUE'] = df['HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS VALUE'].str.lstrip('<>')
df['HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS VALUE'] = df['HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS VALUE'].str.split('%').str[0]

import re
def float_ignore_plus_minus(mynumber):
    mynumber = re.sub(r'[A-Za-z]', '', mynumber)
    try:
        if '-' in mynumber:
            return float(mynumber.split('-')[1])
        else:
            return sum(map(float,mynumber.replace('l', '1').strip('()=≤').split("±")))
    except:
        return float("NaN")

df['HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS VALUE'] = df['HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS VALUE'].apply(float_ignore_plus_minus)

df['SEQUENCE'] = df['SEQUENCE'].str.replace(" ", "")
pepfeature.aa_molecular_weight.calc_df(df, Ncores=1, aa_column='SEQUENCE')
df = df.dropna(subset=['HEMOLITIC CYTOTOXIC ACTIVITY - TARGET CELL', 'HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS GROUP', 'HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS VALUE'])

# display(df)

In [6]:
min_len = 10
max_len = 50

df['len'] = df['SEQUENCE'].str.len()
df = df[df['len'] <= max_len]
df = df[df['len'] >= min_len]
# display(df)

In [7]:
def conc_str_process(concentration_str):
    def float_ignore_plus_minus(mynumber):
        try:
            return sum(map(float,mynumber.split("±")))
        except:
            return float("NaN")

    concentration_str = concentration_str.replace("–","-")
    concentration_str = concentration_str.replace("->","-")
    concentration_str = concentration_str.replace(",",".") 

    if concentration_str[0] == '<':
        if concentration_str[1] == '=':
            concentration_tmp = float_ignore_plus_minus(concentration_str[2:])
        else:
            concentration_tmp = float_ignore_plus_minus(concentration_str[1:])
        concentration = concentration_tmp
    elif concentration_str[0] == '>':
        if concentration_str[1] == '=':
            concentration_tmp = float_ignore_plus_minus(concentration_str[2:])
        else:
            concentration_tmp = float_ignore_plus_minus(concentration_str[1:])
        concentration = concentration_tmp
    elif "-"  in concentration_str:
        concentrations = concentration_str.split("-")
        concentration =  float_ignore_plus_minus(concentrations[0]) + float_ignore_plus_minus(concentrations[1])
        concentration /= 2
    else:
        concentration = float_ignore_plus_minus(concentration_str)

    return concentration

df['HEMOLITIC CYTOTOXIC ACTIVITY - CONCENTRATION'] = df['HEMOLITIC CYTOTOXIC ACTIVITY - CONCENTRATION'].apply(lambda x: conc_str_process(x))
df = df.dropna(subset=['HEMOLITIC CYTOTOXIC ACTIVITY - CONCENTRATION'])

In [8]:
def calc_ugml_concentration(x, val, unit, mw, output):
    if(x[unit] == 'µg/ml'):
        x[output] = x[val]
    else:
        x[output] = x[val] * x[mw] / 1000
    return x

def calc_uM_concentration(x, val , unit, mw, output):
    if(x[unit] == 'ug/ml'):
        x[output] = x[val] / x[mw] * 1000
    else:
        x[output] = x[val]
    return x

df = df.apply(lambda x: calc_ugml_concentration(x, 'HEMOLITIC CYTOTOXIC ACTIVITY - CONCENTRATION', 'HEMOLITIC CYTOTOXIC ACTIVITY - UNIT', 'feat_molecular_weight', 'concentration'), axis=1)
# df = df.apply(lambda x: calc_uM_concentration(x, 'HEMOLITIC CYTOTOXIC ACTIVITY - CONCENTRATION', 'HEMOLITIC CYTOTOXIC ACTIVITY - UNIT', 'feat_molecular_weight', 'concentration'), axis=1)

In [9]:
df = df.drop(['COMPLEXITY', 'HEMOLITIC CYTOTOXIC ACTIVITY - TARGET CELL', 'HEMOLITIC CYTOTOXIC ACTIVITY - CONCENTRATION', 'HEMOLITIC CYTOTOXIC ACTIVITY - UNIT', 'feat_molecular_weight', 'N TERMINUS', 'C TERMINUS', 'HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS GROUP'], axis=1).copy()

In [10]:
# df = df.drop(['ID'], axis=1)
df = df.rename(columns={'SEQUENCE':'sequence', 'HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS VALUE':'lysis'})
df = df[['sequence', 'concentration', 'lysis', 'len']]
df = df.astype({'sequence':'str', 'concentration':'float32', 'lysis':'float32'})
df

,sequence,concentration,lysis,len
113,RVKRVWPLVIRTVIAGYNLYRAIKKK,21.553379,50.000000,26
115,RVKRVWPLVIRTVIAGYNLYRAIKKK,14.368920,15.600000,26
116,RVKRVWPLVIRTVIAGYNLYRAIKKK,28.737841,23.100000,26
117,RVKRVWPLVIRTVIAGYNLYRAIKKK,57.475681,33.099998,26
118,RVKRVWPLVIRTVIAGYNLYRAIKKK,114.951363,43.599998,26
...,...,...,...,...
156819,ALKKFWEKIKSLF,59.333759,5.000000,13
156834,ALGKFWPKIKSLF,224.135681,5.000000,13
156849,ILKKIWKGKKSLF,231.065598,5.000000,13
156864,ILKKIWKPKKSLF,236.193283,5.000000,13


In [11]:
df = df[df['sequence'] != ""]
diff_prot = df.groupby('sequence', as_index=False)
diff_prot.ngroups

4640

In [12]:
proteins = pd.DataFrame(diff_prot.aggregate(list))
proteins

,sequence,concentration,lysis,len
0,AAAAAAAAAAGIGKFLHSAKKFGKAFVGEIMNS,[200.0],[9.0],[33]
1,AAAAAAAAAK,"[100.0, 500.0]","[5.0, 55.0]","[10, 10]"
2,AAAKAALNAVLVGANA,[67.79440307617188],[50.0],[16]
3,AAGKGLVSNLLEK,"[121.25599670410156, 242.51199340820312, 485.0...","[0.0, 28.0, 100.0]","[13, 13, 13]"
4,AAGMGFFGAR,[59.101680755615234],[0.0],[10]
...,...,...,...,...
4635,YYHFWHRGVTKR,[277.20599365234375],[5.0],[12]
4636,YYHFWHRGVTKRSLSPHRPR,[438.48150634765625],[5.0],[20]
4637,YYHFWHRGVTKRSLSPHRPRHSR,[503.64898681640625],[5.0],[23]
4638,YYHFWHRGVTKRSLSPHRPRHSRLQR,[571.3770141601562],[10.0],[26]


In [13]:
proteins['zipped_list'] = proteins.apply(lambda x: sorted(zip(x['concentration'], x['lysis']), key=lambda pair: (pair[0], -pair[1])), axis=1)
# proteins['seq_len'] = proteins['sequence'].str.len()
# proteins['seq_len'].max()   # max len = 133
# proteins

In [14]:
def remove_inverse_pairs(x: list):
    ret = []
    ret.append(x[0])
    for pair in x[1:]:  # (conc, lysis)
        if pair[0] == ret[-1][0]:
            continue
        if pair[0] > ret[-1][0] and pair[1] >= ret[-1][1]:
            ret.append(pair)
    return ret

def select_largest(x: list):
    return x[-1:]

def unzip(x: pd.Series):
    x['concentration'], x['lysis'] = zip(*x['zipped_list'])
    return x

proteins['zipped_list'] = proteins['zipped_list'].apply(lambda x: select_largest(x))
proteins = proteins.apply(lambda x:unzip(x), axis=1)
proteins
proteins = proteins.drop('zipped_list', axis=1)
proteins = proteins.drop('len', axis=1)
proteins

,sequence,concentration,lysis
0,AAAAAAAAAAGIGKFLHSAKKFGKAFVGEIMNS,"(200.0,)","(9.0,)"
1,AAAAAAAAAK,"(500.0,)","(55.0,)"
2,AAAKAALNAVLVGANA,"(67.79440307617188,)","(50.0,)"
3,AAGKGLVSNLLEK,"(485.02398681640625,)","(100.0,)"
4,AAGMGFFGAR,"(59.101680755615234,)","(0.0,)"
...,...,...,...
4635,YYHFWHRGVTKR,"(277.20599365234375,)","(5.0,)"
4636,YYHFWHRGVTKRSLSPHRPR,"(438.48150634765625,)","(5.0,)"
4637,YYHFWHRGVTKRSLSPHRPRHSR,"(503.64898681640625,)","(5.0,)"
4638,YYHFWHRGVTKRSLSPHRPRHSRLQR,"(571.3770141601562,)","(10.0,)"


In [15]:
proteins = proteins.explode(column=['concentration', 'lysis'])
proteins = proteins.astype({'sequence':'str', 'concentration':'float32', 'lysis':'float32'})
display(proteins)

,sequence,concentration,lysis
0,AAAAAAAAAAGIGKFLHSAKKFGKAFVGEIMNS,200.000000,9.0
1,AAAAAAAAAK,500.000000,55.0
2,AAAKAALNAVLVGANA,67.794403,50.0
3,AAGKGLVSNLLEK,485.023987,100.0
4,AAGMGFFGAR,59.101681,0.0
...,...,...,...
4635,YYHFWHRGVTKR,277.205994,5.0
4636,YYHFWHRGVTKRSLSPHRPR,438.481506,5.0
4637,YYHFWHRGVTKRSLSPHRPRHSR,503.648987,5.0
4638,YYHFWHRGVTKRSLSPHRPRHSRLQR,571.377014,10.0


In [16]:
# proteins.dtypes

In [17]:
# proteins = proteins[proteins['lysis'] != 50.0]
# proteins = proteins[proteins['lysis'] != 0]
# proteins = proteins[proteins['lysis'] != 100.0]

In [18]:
proteins['label'] = proteins['lysis'].apply(lambda x: 1.0 if x > 20 else 0.0)
proteins['label'] = proteins['label'].astype('float32')


,sequence,concentration,lysis
label,,,
0.0,2383,2383,2383
1.0,2257,2257,2257


In [19]:
# restrict concentration
proteins = proteins[proteins['concentration'] <= 300]
proteins = proteins[proteins['concentration'] > 0]
c_max = proteins['concentration'].max()
c_min = proteins['concentration'].min()
proteins['concentration'] = proteins['concentration'].apply(lambda x: (x-c_min)/(c_max-c_min))
proteins['concentration'] = proteins['concentration'].astype('float32')
proteins

,sequence,concentration,lysis,label
0,AAAAAAAAAAGIGKFLHSAKKFGKAFVGEIMNS,0.666444,9.0,0.0
2,AAAKAALNAVLVGANA,0.225465,50.0,1.0
4,AAGMGFFGAR,0.196470,0.0,0.0
7,AAKIILNPKFR,0.618715,10.0,0.0
8,AAKIILNPKFRCKAAFC,0.232298,10.0,0.0
...,...,...,...,...
4624,YNYSYIITRS,0.239748,2.0,0.0
4625,YPCKLNLKLGKVPFH,0.271786,0.0,0.0
4630,YSKSLPLSVLNP,0.853236,5.0,0.0
4635,YYHFWHRGVTKR,0.923969,5.0,0.0


In [23]:
group = proteins.groupby('label')
group.count()

,sequence,concentration,lysis
label,,,
0.0,1572,1572,1572
1.0,1385,1385,1385


In [20]:
train, test = train_test_split(proteins, test_size=0.1, random_state=0)

In [25]:
group = train.groupby('label')
group.count()

,sequence,concentration,lysis
label,,,
0.0,1417,1417,1417
1.0,1244,1244,1244


In [21]:
proteins.to_parquet('peptides.parquet')
train.to_parquet('train.parquet')
test.to_parquet('test.parquet')

In [22]:
# with pd.option_context('display.max_rows', None,
#                        'display.max_columns', None,
#                        'display.precision', 3,):
#      display(proteins)

### Modify the isHemo function to adjust the thresholds